In [1]:
import h5py
import numpy as np
from PIL import Image, ImageOps, ImageChops
import PIL
import matplotlib.pyplot as plt
import os
import time

In [2]:
PIL.Image.MAX_IMAGE_PIXELS = None

In [5]:
original_image = Image.open('/data/data/different_threshold_images/original.png').convert('RGB')

In [3]:
def bw_to_transparent_color(image, color='blue'):
    # Convert the image to grayscale and then to RGBA
    bw_image = image.convert("RGBA")

    # Define the color mappings
    color_mapping = {
        'blue': (0, 0, 255, 255),
        'red': (255, 0, 0, 255),
        'purple':(128, 0, 128) 
    }

    # Get the selected color or default to blue
    selected_color = color_mapping.get(color, (0, 0, 255, 255))

    # Create a new image with the same size and RGBA mode
    transparent_color_image = Image.new("RGBA", bw_image.size)

    # Process each pixel
    for x in range(bw_image.width):
        for y in range(bw_image.height):
            r, g, b, a = bw_image.getpixel((x, y))
            # White (255, 255, 255) becomes transparent
            if r == 255:
                transparent_color_image.putpixel((x, y), selected_color)
            # Black (0, 0, 0) becomes the selected color
            else:
                transparent_color_image.putpixel((x, y), (0, 0, 0, 0))

    return transparent_color_image

In [ ]:
#for threshold in [0.25, 0.5, 0.75]
threshold = 0.9
fracture_true = Image.open('data/data/different_threshold_images/fracture_image_orig.png').convert('1')
fracture_threshold = Image.open(f'data/data/different_threshold_images/fracture_image_{threshold}.png').convert('1')
t0 = time.time
false_negatives = ImageChops.difference(fracture_true, fracture_threshold)
false_positives = ImageChops.difference(fracture_threshold, fracture_true)
intersection = ImageChops.logical_and(fracture_true, fracture_threshold) 
t1 = time.time
#print(t1-t0)
plt.figure(figsize = (15,5))
plt.imshow(false_negatives)

plt.figure(figsize = (15,5))
plt.imshow(false_positives)

plt.figure(figsize = (15,5))
plt.imshow(intersection)
t2 = time.time
false_negatives = bw_to_transparent_color(false_negatives, "blue")
false_negatives = bw_to_transparent_color(false_positives, "red")
false_negatives = bw_to_transparent_color(intersection, "purple")
t3 = time.time
#print(t3-t2)
plt.figure(figsize = (15,5))
plt.imshow(false_negatives)

plt.figure(figsize = (15,5))
plt.imshow(false_positives)

plt.figure(figsize = (15,5))
plt.imshow(intersection)

'''

false_negatives = false_negatives.convert('L')
false_positives = false_positives.convert('L')
intersection = intersection.convert('L')

# Convert the original image to RGBA
original_rgba = original_image.convert('RGBA')

# Overlay the differences and intersection on the original image
combined = Image.alpha_composite(original_rgba, false_negatives_rgb)
combined = Image.alpha_composite(combined, false_positives_rgb)
combined_with_intersection = Image.alpha_composite(combined, intersection_rgba)

# Save and display the result
combined_with_intersection.save(f'data/data/different_threshold_images/combined_image_{threshold}.png')
#combined_with_intersection.show()

# Save the intersection separately
#intersection_rgba.save('data/data/different_threshold_images/intersection_image.png')
#intersection_rgba.show()
'''

In [ ]:
for threshold in [0.25, 0.5, 0.75]:
    fracture_true = Image.open('data/data/different_threshold_images/fracture_image_orig.png').convert('1')
    fracture_threshold = Image.open(f'data/data/different_threshold_images/fracture_image_{threshold}.png').convert('1')

    false_negatives = ImageChops.difference(fracture_true, fracture_threshold)
    false_positives = ImageChops.difference(fracture_threshold, fracture_true)
    intersection = ImageChops.logical_and(fracture_true, fracture_threshold) 

    fracture_true = 0
    fracture_threshold = 0
    def colorize_with_transparency(diff, color):
        # Create an RGBA image where the alpha channel is based on the difference mask
        rgba_image = Image.new("RGBA", diff.size)
        colored_image = ImageOps.colorize(diff, black="black", white=color)

        # Use the diff itself as the alpha mask by converting it to "L" mode
        alpha_mask = diff.point(lambda p: p > 0 and 255)

        # Paste the colored image onto the RGBA image using the alpha mask
        rgba_image.paste(colored_image, (0, 0), alpha_mask)
        return rgba_image


    # Define colors for differences and intersection
    color_diff1 = (255, 0, 0)  # Red
    color_diff2 = (0, 0, 255)  # Blue
    color_intersection = (128, 0, 128)  # Purple

    false_negatives = false_negatives.convert('L')
    false_positives = false_positives.convert('L')
    intersection = intersection.convert('L')

    # Apply colors and transparency
    false_negatives = colorize_with_transparency(false_negatives, color_diff1)
    false_positives = colorize_with_transparency(false_positives, color_diff2)
    intersection = colorize_with_transparency(intersection, color_intersection)

    # Convert the original image to RGBA
    original_rgba = original_image.convert('RGBA')

    # Overlay the differences and intersection on the original image
    combined = Image.alpha_composite(original, false_negatives)
    combined = Image.alpha_composite(combined, false_positives)
    combined_with_intersection = Image.alpha_composite(combined, intersection)

    # Save and display the result
    combined_with_intersection.save(f'data/data/different_threshold_images/combined_image_{threshold}.png')
    #combined_with_intersection.show()

    # Save the intersection separately
    #intersection_rgba.save('data/data/different_threshold_images/intersection_image.png')
    #intersection_rgba.show()